In [79]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import plotly.express as px
from selenium import webdriver

In [80]:
driver = webdriver.Chrome()

### Fonction pour générer l'URL en fonction du terme recherché

In [81]:
def url_by_term(search_term,num_page):
    form = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1&page="+str(num_page)
    term = search_term.replace(" ", "+")
    return form.format(term)

term = "ordinateur portable"


### Extraction des liens

In [82]:
links = []
for i in range(20):
    
    url = url_by_term(term,i)
    driver.get(url)

    bs = BeautifulSoup(driver.page_source,'html.parser')
    results = bs.find_all('h2', {'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-2'})
    

    for r in results:
        links.append('https://www.amazon.com'+r.find('a')['href'])


### Extraction d'avis et de commentaire

In [83]:

for j in range(len(links)):
    links[j] = links[j].replace("dp","product-reviews")
    
review = []    
list_links=[]
positif=[]
for g in links:
    for i in range(5):
        for j in ['positive','critical']:
            list_links.append(g+"&filterByStar="+j+"&pageNumber="+str(i))
            driver.get(g+"&filterByStar="+j+"&pageNumber="+str(i))

            bs = BeautifulSoup(driver.page_source,'html.parser')
            results = bs.find_all('span', {'data-hook':'review-body'})
    

            for r in results:
                review.append(r.find('span').text)
                
                
                if j == 'positive':
                    positif.append(1)
                else:
                    positif.append(0)
                


### Nettoyage et Mise en Format JSON

In [84]:

d = {'review' : review, 'positive': positif}
review_df = pd.DataFrame(data=d)
review_df = review_df.replace('\n','', regex=True)
review_df = review_df.replace('\u2019','`', regex=True)
review_df = review_df.replace('\u201d','', regex=True)
review_df = review_df.replace('\u201c','', regex=True)
review_df = review_df.replace('\/',' ', regex=True)


review_df.to_json(r'C:\Users\cleme\OneDrive\Documents\Python Scripts\Amazon Web Scrapping\review_amazon_'+term.replace(' ','')+'.json',orient='records')